# BUFR Message: Decoding BUFR 

This notebook demonstrates how to decode BUFR message using the `eccodes` library.




## Explore a BUFR message with comand bufr_ls

### bufr_ls -p masterTablesVersionNumber bitmap.bufr 

>  masterTablesVersionNumber   
     39                                               
       
### bufr_ls -p center,localTablesVersionNumber bitmap.bufr
> center            localTablesVersionNumber   
     ecmwf                        0

## Explore a BUFR message with comand bufr_dump

### bufr_dump -p  bitmap.bufr 

>dataCategory=5 <br>
internationalDataSubCategory=255 <br>
dataSubCategory=0 <br>
masterTablesVersionNumber=39 <br>
localTablesVersionNumber=0 <br>
typicalYear=2025 <br>
typicalMonth=11 <br>
typicalDay=9 <br>
typicalHour=11 <br>
typicalMinute=55 <br>
typicalSecond=5 <br>
numberOfSubsets=1 <br>
observedData=1 <br>
compressedData=1 <br>
unexpandedDescriptors={
      001001, 001002, 004001, 004002, 004003, 004004, 004005, 005001, 006002, 011001, 
      011002, 222000, 236000, 101004, 031031, 033007  }





## Setup enviroment
First, we'll install and import the required libraries.

In [1]:
from eccodes import *
import numpy as np
import pandas as pd
import os

## Welcome to BUFR world
These functions will help us process the BUFR messages.

In [16]:
def read_bufr_message(file_path):
    """
    Read a BUFR message from a file.
    
    Args:
        file_path (str): Path to the BUFR file
                
    Returns:
        eccodes.BufrMessage: BUFR message object
    """
    with open(file_path, 'rb') as f:
        bufr = eccodes.codes_bufr_new_from_file(f)
        if bufr is None:
            raise ValueError(f"Failed to load BUFR message {message_number} from {file_path}")
        return bufr

def decode_bufr_message(bufr):
    """
    Decode a BUFR message and extract key metadata and observations.
    
    Args:
        bufr: BUFR message object
        
    Returns:
        dict: Dictionary containing decoded data
    """
    # Decode the message
    eccodes.codes_set(bufr, 'unpack', 1)
      
    
    # Extract  meteorological parameters
    try:
        data = {
            'masterTablesVersionNumber': eccodes.codes_get(bufr, 'masterTablesVersionNumber'),
            'dataCategory': eccodes.codes_get(bufr, 'dataCategory'),
            'typicalDate': eccodes.codes_get(bufr, 'typicalDate'),
            'typicalTime': eccodes.codes_get(bufr, 'typicalTime'),
            'latitude': eccodes.codes_get(bufr, 'latitude'),
            'longitude': eccodes.codes_get(bufr, 'longitude'),           
            'wind direction': eccodes.codes_get(bufr, 'windDirection'),
            'wind direction percent confidence': eccodes.codes_get(bufr, 'windDirection->percentConfidence'),
            'wind speed': eccodes.codes_get(bufr, 'windSpeed'),
        }
    except Exception as e:
        print(f"Warning: Not all parameters could be extracted: {str(e)}")
          
  
    return {'data': data}



## Process BUFR File
Now we can process a BUFR file and analyze its contents.

In [18]:
# Specify your BUFR file path
bufr_file="/Users/mamc/tutorial/bin/training-bufr/bitmap.bufr"

# Read and decode the BUFR message
bufr = read_bufr_message(bufr_file)

# Decode the BUFR message
decoded_data = decode_bufr_message(bufr)


# Convert decoded BUFR data into a pandas DataFrame
df = pd.DataFrame(decoded_data)

display(df)

# Decode Bit-Map values
#bitmap = eccodes.codes_get_array(bufr, 'dataPresentIndicator')
#print("Bit-Map  ",bitmap)

# Clean up
eccodes.codes_release(bufr)

,data
masterTablesVersionNumber,39
dataCategory,5
typicalDate,20251009
typicalTime,115505
latitude,50.07
longitude,14.43
wind direction,208
wind direction percent confidence,82
wind speed,10.0


## Quality information using Bitmap
### Let's undersand meaning of these attributed to BUFR meteorological variables.
 We look first at the BUFR template definition. We can find this information in unexpandedDescriptors. 
  

222000 - Quality information to follow for the elements in class 33 <br>
236000 - Define data present bit-map <br> 
101004 - Replicate 1 descriptor 4 times <br>
031031 - Data present indicator <br>
033007 - Per centage confidence <br>

Element **031031 - Data present indicator** is defined as 0 or 1. <br>
We decode its value with
>  **codes_get(bufr, 'dataPresentIndicator')**

The values in our BUFR file   
> 	1 1 0 1
    0 = Data present
    1 = Data not present; 

To obtain info of quality control indicator related to windDirection
>**codes_get(bufr, 'windDirection->percentConfidence)**

The value in our BUFR file is **82**, which correspond to 
> 82 % ;


## Decode BUFR File
Using bufr_dump - Dpython 

> bufr_dump -Dpython my.bufr >sample_decode.py  